<a href="https://colab.research.google.com/github/Mirucla/Applied-Data-Management/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
     

# 1) Import the Credit Card Fraud Data From CCLE

In [20]:
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [21]:
df = pd.read_csv("/content/gdrive/MyDrive/fraudTest.csv")

In [22]:
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [23]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

# 2) Select four columns to use as features (one just be trans_date_trans)

In [24]:
df_select= df[['trans_date_trans_time', 'gender', 'amt', 'is_fraud']]
df_select

,trans_date_trans_time,gender,amt,is_fraud
0,2020-06-21 12:14:25,M,2.86,0
1,2020-06-21 12:14:33,F,29.84,0
2,2020-06-21 12:14:53,F,41.28,0
3,2020-06-21 12:15:15,M,60.05,0
4,2020-06-21 12:15:17,M,3.19,0
...,...,...,...,...
555714,2020-12-31 23:59:07,M,43.77,0
555715,2020-12-31 23:59:09,M,111.84,0
555716,2020-12-31 23:59:15,F,86.88,0
555717,2020-12-31 23:59:24,M,7.99,0


# 3) Create a unique variable out of trans_date_trans_time

In [26]:
type(df_select['trans_date_trans_time'][0])
     

str

In [27]:
df_select['trans_date_trans_time'] = pd.to_datetime(df_select['trans_date_trans_time'])

<ipython-input-27-035beec6fe23>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['trans_date_trans_time'] = pd.to_datetime(df_select['trans_date_trans_time'])


In [28]:
df_select['hour']=[i.hour for i in df_select['trans_date_trans_time']]
df_select['hour'].head()

<ipython-input-28-b9add3657b11>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['hour']=[i.hour for i in df_select['trans_date_trans_time']]


0    12
1    12
2    12
3    12
4    12
Name: hour, dtype: int64

Classifying the minutes by day and night 

In [29]:
df_select['Day'] = np.where(df_select['hour']>=12, 1,0)
df_select['Night'] = np.where(df_select['hour']<12, 1,0)
df_select.drop('trans_date_trans_time', axis =1)
dummies = pd.get_dummies(df['gender'])
mergedf = pd.concat([df_select,dummies], axis=1)
df_select=mergedf.drop('gender',axis=1)
     

<ipython-input-29-d8e9444ec72c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['Day'] = np.where(df_select['hour']>=12, 1,0)
<ipython-input-29-d8e9444ec72c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['Night'] = np.where(df_select['hour']<12, 1,0)


# 4) Train a Logistic regression

In [33]:
train_size = int(len(df_select)*0.80)
target =df_select["is_fraud"]

In [32]:
features = df_select.drop(["is_fraud"], axis = 1)

In [34]:
in_target = target[:train_size]
out_target=target[train_size:]
in_data=features[:train_size]
out_data=features[train_size:]

In [35]:
mod = LogisticRegression().fit(in_data, in_target)

# 6)  The company you are working for wants to target at a False Positive rate of 5% what threshold should you use? (Use oversampled data)

In [36]:
from sklearn.metrics import confusion_matrix


In [37]:
confi = LogisticRegression()
confi.fit(in_data, in_target)

LogisticRegression()

In [38]:
probability = confi.predict_proba(out_data)

In [39]:
Threshhold=0

In [40]:
for t in np.arange(0, 1, 0.01):
    predicts = probability[:,1] > t
    cm = confusion_matrix(out_target, predicts)
    fpr = cm[0,1] / (cm[0,0] + cm[0,1])
    if fpr <= 0.05:
        threshold = t
        break

In [41]:
threshold

0.01

Therefore, the threshold is 10%. 

# 7) If the company makes .02*amt on True transactions and loses -amt on False (Use original data)

In [46]:
out_data["Predicts"] = predicts
out_data["TP"] = out_data["Predicts"] * out_target

<ipython-input-46-0480278957d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_data["Predicts"] = predicts
<ipython-input-46-0480278957d7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_data["TP"] = out_data["Predicts"] * out_target


In [47]:
out_data["TP"]

444575    0
444576    0
444577    0
444578    0
444579    0
         ..
555714    0
555715    0
555716    0
555717    0
555718    0
Name: TP, Length: 111144, dtype: int64

In [48]:
df_compare = pd.concat([out_data, out_target], axis = 1)
df_compare['Predicts'] = [0 if i == False else 1 for i in df_compare['Predicts']]
     

In [49]:
df_compare['FN']= [0 if i==False else 1 for i in (df_compare['is_fraud']==1)&(df_compare['Predicts']==0)]
df_compare['TP']= out_data["TP"]

In [50]:
df_compare['Profit(+)/Loss(-)']= 0.02*df_compare['amt']*df_compare["TP"] + df_compare['FN']*-df_compare['amt']
df_compare['Profit(+)/Loss(-)'].sum()

-14888.219200000001

In [51]:
# Therefore the loss will be 14,888 if logistic regression is used 

# 8) Using Logistic Regression Lasso to inform you. Would you use the selected features in a trusted prediction model?

In [57]:
mod = LogisticRegression(solver='liblinear', penalty='l1').fit(in_data, in_target)
mod.coef_
     

array([[ 2.16182578e-03,  1.17961543e-01, -4.54186017e+00,
        -3.01837455e+00, -1.98850946e+00, -2.00379061e+00]])

In [ ]:
# Selected Features are okay for using in the trusted prediction model